In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
#| export
from fastcore.utils import *
from fastcore.xml import *
from fasthtml.common import *
from fasthtml.svg import *
from json import loads
import pathlib

In [ ]:
__file__='../fastlucide/_icons.py'

In [ ]:
#| export
def read_icons():
    path = pathlib.Path(__file__).parent
    fpath = path/'_icons.py'
    return loads(fpath.read_text())

In [ ]:
icons = read_icons()

In [ ]:
nm = 'a-arrow-down'
ico = icons[nm]
ico

[['path', {'d': 'M3.5 13h6'}],
 ['path', {'d': 'm2 16 4.5-9 4.5 9'}],
 ['path', {'d': 'M18 7v9'}],
 ['path', {'d': 'm14 12 4 4 4-4'}]]

In [ ]:
svg_attrs = dict(fill='none', stroke='currentColor', stroke_width='2', stroke_linecap='round', stroke_linejoin='round')

In [ ]:
#| export
def sz_attrs(sz):
    "Attrs necessary to size an SVG"
    return dict(viewBox=f'0 0 24 24', width=f'{sz}px', height=f'{sz}px')

In [ ]:
sym = [ft(t, **attrs) for t,attrs in ico]
show(Svg(*sym, **sz_attrs(24), **svg_attrs))
show(Svg(*sym, **sz_attrs(16), **svg_attrs))

In [ ]:
#| export
def symbol(
    icons, # icon dict
    nm, # Name of icon in lucide
    pre='' # Prefix to add to element id
):
    "Create a `symbol` element for an icon"
    ico = icons[nm]
    sym = [ft(t, **attrs) for t,attrs in ico]
    return Symbol(*sym, id=pre+nm)

In [ ]:
symbol(icons, nm, 'l-')

```html
<symbol id="l-a-arrow-down"><path d="M3.5 13h6"></path><path d="m2 16 4.5-9 4.5 9"></path><path d="M18 7v9"></path><path d="m14 12 4 4 4-4"></path></symbol>
```

In [ ]:
#| export
def sprites(
    icons, # icon dict
    nms, # List of lucide icon names
    pre='' # Prefix to add to all element ids
):
    "SVG element containing all symbols in `nms`"
    syms = [symbol(icons, nm, pre) for nm in nms]
    return Svg(Defs(*syms), style="display: none")

In [ ]:
sps = sprites(icons, [nm, 'accessibility'], 'l-')
show(sps)

In [ ]:
#| export
def icon(
    nm, # Name of icon in lucide
    pre='', # Prefix to add to element id
    cls="lucide-icon", # class to use for svg
    sz=24, # size of svg
    **attrs # additional attrs for svg
):
    "A `use` element in an `svg` element refering to `nm`"
    return Svg(Use(href=f"#{pre}{nm}"), cls=cls, icon=nm, **sz_attrs(sz), **attrs)

In [ ]:
used = icon('accessibility', 'l-', sz=16, **svg_attrs)
used

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="16px" width="16px" icon="accessibility" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" class="lucide-icon"><use href="#l-accessibility"></use></svg>
```

In [ ]:
show(used)

In [ ]:
#| export
def SvgStyle(cls="lucide-icon"):
    "Styles required for lucide icons to display correctly"
    return Style(f'.{cls} {{ stroke: currentColor; fill: none; stroke-width: 2; stroke-linecap: round; stroke-linejoin: round; }}')

In [ ]:
#| export
class SvgSprites:
    "Create an track used icons"
    def __init__(self, pre='', sz=24, cls="lucide-icon", nms=(), **attrs):
        nms = set(nms)
        self.attrs = attrs
        self.icons = read_icons()
        store_attr()

    def __call__(self, nm, sz=None, cls="", **kw):
        self.nms.add(nm)
        if not sz: sz=self.sz
        attrs = self.attrs | kw
        return icon(nm, self.pre, sz=sz, cls = f"{self.cls} {cls}", **attrs)

    def __ft__(self):
        return SvgStyle(cls=self.cls),sprites(self.icons, self.nms, self.pre)

In [ ]:
ss = SvgSprites('lc-', sz=18)
ss(nm)

```html
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" height="18px" width="18px" icon="a-arrow-down" class="lucide-icon "><use href="#lc-a-arrow-down"></use></svg>
```

In [ ]:
show(ss('a-arrow-down'))
show(ss('accessibility'))
show(ss)

In [ ]:
show(Style('.hover-bg:hover { background-color: aliceblue; }'))
show(ss('accessibility', sz=30, cls='hover-bg'))

In [ ]:
show(Div(ss('accessibility'), style='background-color: aliceblue; color:blue'))

In [ ]:
#| export
trans = {'side-by-side': 'translate(24, 0)', 'stacked': 'translate(0, 24)'}
def combined(
    icon1, icon2, # Icon partials from SvgSprites
    position='overlap', # 'overlap', 'side-by-side', or 'stacked'
    scale=None, # Scale factor (auto: 0.5 for positioned, 1.0 for overlap)
    **kw # Additional attrs for svg
):
    "Combine two icons with positioning"
    icon1(), icon2()  # Register with spritesheet
    ss = icon1.func
    nm1,nm2 = icon1.args[0], icon2.args[0]
    
    # Set scale and translation based on position
    if scale is None: scale = 1. if position=='overlap' else 0.5
    t1,t2 = f'scale({scale})', f'scale({scale}) {trans.get(position, 'translate(0, 0)')}'
    return Svg(Use(href=f"#{ss.pre}{nm1}", transform=t1),
               Use(href=f"#{ss.pre}{nm2}", transform=t2),
               cls=ss.cls, icon=f'{nm1}-{nm2}', 
               **sz_attrs(kw.pop('sz', ss.sz)), **ss.attrs, **kw)

In [ ]:
A_arrow_down, Accessibility = partial(ss, 'a-arrow-down'), partial(ss, 'accessibility')
show(combined(A_arrow_down, Accessibility))

In [ ]:
show(combined(A_arrow_down, Accessibility, position='side-by-side', sz=32))

In [ ]:
show(combined(A_arrow_down, Accessibility, position='stacked', sz=32))

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()